In [5]:
import tpc5
import h5py
import numpy as np

/home/jes-lab/anaconda3/envs/obspy/lib/python3.6/site-packages/h5py/__init__.py:36: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters


In [7]:
f = h5py.File("020918_shear/run.tpc5", "r")

In [35]:
# all channels have the same number of blocks, use channel 1
cg = f[tpc5.getChannelGroupName(1)]
len(cg['blocks'].keys())

35

In [ ]:
# to view the keys of an hdf5 file:
list(f.keys())

# to then access one of the keys:
f['key']